In [5]:
# imports
import openai
import os
import sys
import langchain
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import pprint

In [2]:
# pip installs
# !pip install langchain
# !pip install torch
# !pip install transformers
# !pip install faiss-cpu
# !pip install pypdf
# !pip install sentence-transformers
# !pip install unstructured==0.7.12   # has to be this downgraded version 
# !pip install 'unstructured[pdf]'
# !pip install tiktoken
# !pip install huggingface_hub
# from huggingface_hub import login

In [3]:
# Load in collected pdfs
loader = DirectoryLoader('/Users/cstone/Documents/Projects/LUCID/protocol_pdfs/all_pdfs_test', glob='**/*.pdf', show_progress=True)
documents = loader.load()

# Q: Can the papers be separated into separate directories and still be loaded in? If so, how?
# Q: What exactly is happening here? The text from the pdfs is being saved to memory?

100%|█████████████████████████████████████████| 246/246 [07:50<00:00,  1.91s/it]


In [7]:
# split loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)

print("done splitting")

done splitting


In [8]:
# Embed the chunked texts using a Transformer
from langchain.embeddings import HuggingFaceEmbeddings
modelPath = "sentence-transformers/all-MiniLM-l6-v2"
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings':False}
embeddings = HuggingFaceEmbeddings(
  model_name = modelPath,
  model_kwargs = model_kwargs,
  encode_kwargs=encode_kwargs
)

In [9]:
# create a vector database
from langchain.vectorstores import FAISS
db = FAISS.from_documents(docs, embeddings)

In [10]:
# Run a similarity search between query and documents in vecor database

# # Example
# question = "What is CRISPR?"
# searchDocs = db.similarity_search(question)
# print(searchDocs[0].metadata)

# Navigating searchDocs
# print(searchDocs[0].metadata)  
# print(searchDocs[0].page_content)
# pprint.pp(doc_attributes)  # to figure out what attributes are availible in the search docs

In [13]:
# Collect text from one of Rick's hypotheses and run similarity search to find most related paper
test_hypothesis_path = Path("/Users/cstone/Documents/Projects/LUCID/lucid_protocols/00a06d706e56c64d47ee6b06bd59595268839391.txt_chunk_1_.report")
test_hypothesis_contents = test_hypothesis_path.read_text()
# print(test_hypothesis_contents)

searchDocs = db.similarity_search(test_hypothesis_contents)
print(f"1: {searchDocs[0].metadata}")
print(f"2: {searchDocs[1].metadata}")
print(f"3: {searchDocs[2].metadata}")
print(f"4: {searchDocs[3].metadata}")
# print(f"5: {searchDocs[4].metadata}")  # Q: How is this out of range?

    

1: {'source': '/Users/cstone/Documents/Projects/LUCID/protocol_pdfs/all_pdfs_test/a-novel-method-for-the-isolation-of-single-cells-m-c3szynf6.pdf'}
2: {'source': '/Users/cstone/Documents/Projects/LUCID/protocol_pdfs/all_pdfs_test/drug-sensitivity-assays-of-human-cancer-organoid-c-bpsgmnbw.pdf'}
3: {'source': '/Users/cstone/Documents/Projects/LUCID/protocol_pdfs/all_pdfs_test/drug-sensitivity-assays-of-human-cancer-organoid-c-bpsgmnbw.pdf'}
4: {'source': '/Users/cstone/Documents/Projects/LUCID/protocol_pdfs/all_pdfs_test/fragment-analysis-guide-technote.pdf'}
